In [ ]:
from fastapi import FastAPI
import pickle
import numpy as np
import os

model_path = "ml_model.pkl" 
scaler_path = "scaler.pkl"

if not os.path.exists(model_path) or not os.path.exists(scaler_path):
    raise FileNotFoundError("Model or Scaler file not found! Please train the model first.")

with open(model_path, "rb") as f:
    model = pickle.load(f)
with open(scaler_path, "rb") as f:
    scaler = pickle.load(f)

app = FastAPI()

@app.post("/predict/")
def predict(transaction: dict):
    # Extract & scale the transaction features
    features = np.array([transaction["features"]]).reshape(1, -1)
    features[:, -1] = scaler.transform(features[:, -1].reshape(-1, 1))  # Scale 'Amount'

    # Predict fraud probability
    prediction = model.predict(features)
    return {"fraud": int(prediction[0])}
